In [2]:
from os import listdir
from os.path import join
import os.path

from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchsummary import summary

import numpy as np
import matplotlib.pyplot as plt

import random

In [3]:
import gdown
import patoolib

url       = 'https://drive.google.com/uc?id='
file_id   = '1iVt97n4fpFKUeTW93kNGMzbadRbHCjUf'
data_path = '../data/'
data_dir  = 'extracted_images'
data_file = 'math_data.rar'

if os.path.isdir(data_path+data_dir):
    print('already exist')

elif not os.path.isfile(data_path+data_file):
    gdown.download(url + file_id, data_path+data_file, quiet=False)
    patoolib.extract_archive(data_path+data_file, outdir='../data/')
    
else:
    patoolib.extract_archive(data_path+data_file, outdir='../data/')


already exist


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'using {device}')

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

training_epochs = 15
batch_size = 50

using cuda


In [13]:
trans = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor() 
    ])
trainset = torchvision.datasets.ImageFolder(root = data_path + 'extracted_images',
                                            transform = trans)
classes = trainset.classes
len(trainset.classes)

print(trainset.classes[63])
print(len(trainset.targets))

pi
375974


In [22]:
train_size = int(0.9 * len(trainset))
test_size = len(trainset) - train_size

train_data, test_data = torch.utils.data.random_split(trainset, [train_size, test_size])

train_loader = DataLoader(train_data,
                         batch_size = batch_size,
                         shuffle = True,
                         drop_last = True)

test_loader = DataLoader(test_data,
                         batch_size = batch_size,
                         shuffle = False,
                         drop_last = True)

print(f'train size = {train_size}\ntest size = {test_size}')
print('')
print(f'training data set = {len(train_loader)}\ntest data set = {len(test_loader)}')


images, labels = next(iter(train_loader))
images.shape, labels.shape

train size = 338376
test size = 37598

training data set = 6767
test data set = 751


(torch.Size([50, 1, 45, 45]), torch.Size([50]))

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 4, 1) # input, output, kernel_size, stride
        self.conv2 = nn.Conv2d(20, 50, 4, 1)
        self.fc1 = nn.Linear(9*9*50, 500)
        self.fc2 = nn.Linear(500, 82)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 9*9*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
model = Net()
model.to(device)
summary(model, input_size=(1, 45, 45))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 20, 42, 42]             340
            Conv2d-2           [-1, 50, 18, 18]          16,050
            Linear-3                  [-1, 500]       2,025,500
            Linear-4                   [-1, 82]          41,082
Total params: 2,082,972
Trainable params: 2,082,972
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.40
Params size (MB): 7.95
Estimated Total Size (MB): 8.35
----------------------------------------------------------------


In [9]:
# define the negative log-likelihood loss
loss_func = nn.NLLLoss(reduction='sum')

# define the Adam optimizer
opt = optim.Adam(model.parameters(), lr=1e-4)


In [10]:
# define a helper function to compute the loss value per mini-batch
def loss_batch(loss_func, xb, yb, yb_h, opt=None):
    # obtain loss
    loss = loss_func(yb_h, yb)
    # obtain performance metric
    metric_b = metrics_batch(yb, yb_h)
    if opt is not None:
        loss.backward() # compute gradient
        opt.step() # update parameters
        opt.zero_grad() # set gradients to zero
    return loss.item(), metric_b
    
# define a helper function to compute the accurary per mini-batch
def metrics_batch(target, output):
    # optain output class
    pred = output.argmax(dim=1, keepdim=True)
    # compare output class with target class
    corrects = pred.eq(target.view_as(pred)).sum().item()
    
    return corrects
    
# define a helper fuction to compute the loss and metric values for a dataset
def loss_epoch(model, loss_func, dataset_dl, opt=None):
    loss = 0.0
    metric = 0.0
    len_data = len(dataset_dl.dataset)
    
    for xb, yb in dataset_dl:
        xb = xb.type(torch.float).to(device)
        yb = yb.to(device)
        # obtain model output
        yb_h = model(xb)
        
        loss_b, metric_b = loss_batch(loss_func, xb, yb, yb_h, opt)
        loss += loss_b
        if metric_b is not None:
            metric += metric_b
            
    loss /= len_data
    metric /= len_data
    return loss, metric

In [11]:
# define train_val function
def train_val(epochs, model, loss_func, opt, train_dl, val_dl):
    for epoch in range(epochs):
        model.train() # convert to train mode
        train_loss, train_metric = loss_epoch(model, loss_func, train_dl, opt)
        model.eval() # convert to evaluation mode
        with torch.no_grad():
            val_loss, val_metric = loss_epoch(model, loss_func, val_dl)
        accuracy = 100 * val_metric
        print('epoch: %d, train loss: %.6f, val loss: %.6f, accuracy: %.2f' %(epoch, train_loss, val_loss, accuracy))

In [12]:
train_val(training_epochs, model, loss_func, opt, train_loader, test_loader)

epoch: 0, train loss: 0.837358, val loss: 0.409150, accuracy: 88.46
epoch: 1, train loss: 0.324404, val loss: 0.270938, accuracy: 92.03
epoch: 2, train loss: 0.224117, val loss: 0.197805, accuracy: 94.14
epoch: 3, train loss: 0.168751, val loss: 0.158744, accuracy: 95.16
epoch: 4, train loss: 0.132057, val loss: 0.130260, accuracy: 95.98
epoch: 5, train loss: 0.105312, val loss: 0.105669, accuracy: 96.70
epoch: 6, train loss: 0.086337, val loss: 0.088864, accuracy: 97.07
epoch: 7, train loss: 0.071491, val loss: 0.073947, accuracy: 97.61
epoch: 8, train loss: 0.061405, val loss: 0.067430, accuracy: 97.72
epoch: 9, train loss: 0.053484, val loss: 0.055967, accuracy: 98.20
epoch: 10, train loss: 0.047045, val loss: 0.050081, accuracy: 98.24
epoch: 11, train loss: 0.042382, val loss: 0.044860, accuracy: 98.47
epoch: 12, train loss: 0.038655, val loss: 0.044155, accuracy: 98.49
epoch: 13, train loss: 0.035269, val loss: 0.037002, accuracy: 98.85
epoch: 14, train loss: 0.032487, val loss: 0

In [13]:
torch.save(model.state_dict(), "../model/test.pth")